In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('Data/pca_projected_266_from_resnet_train.csv')
#train_df.reset_index(drop=True, inplace=True)
train_df.head(5)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_257,f_258,f_259,f_260,f_261,f_262,f_263,f_264,f_265,is_iceberg
0,-1.078176,0.358596,-0.215716,-0.372415,-1.290822,0.039064,0.134564,0.787661,-0.613845,0.108084,...,0.268202,-0.910513,0.280417,0.250057,-0.042847,0.724692,-0.741978,-0.324261,0.286298,0
1,-1.273724,1.087702,-1.793496,-1.264303,0.042096,0.182288,0.078520,0.412620,0.980071,-1.091136,...,-0.753098,-0.415596,-0.035685,-1.136880,-0.160498,-0.886964,-0.399335,-2.055109,0.922215,0
2,0.481582,-1.725730,0.391214,-1.857765,0.999916,-0.145043,-0.434842,-0.503897,-0.264354,0.272160,...,1.063322,-0.392458,1.106920,-0.279701,1.667911,1.040562,0.447926,-1.486754,-0.780378,1
3,-1.089239,0.445182,0.128932,0.059036,-0.324501,0.164366,0.122997,1.574274,-0.467762,0.552849,...,1.085721,-1.308412,-0.889094,1.183155,-1.483414,0.219054,-0.598839,-0.953547,-0.252878,0
4,-1.038766,0.221048,-0.418525,0.133892,-0.974831,-0.889494,0.028311,1.753731,-0.450525,0.435784,...,-0.541727,0.168227,0.190182,1.246043,0.217847,0.486931,-0.417607,0.094914,-1.127213,0


In [3]:
features = train_df.columns.tolist()
features.remove('is_iceberg')

train_X = train_df[features]
train_y = np.array(train_df['is_iceberg']).reshape((train_X.shape[0],))

print(train_X.shape, train_y.shape)

(1604, 266) (1604,)


In [4]:
from sklearn.model_selection import GridSearchCV

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier
import xgboost
import lightgbm

In [172]:
lr = LogisticRegression()
params = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'penalty': ['l2'],
    'random_state': [1,2,3,4,5],
    'max_iter': [10000]
}

gs = GridSearchCV(lr, params, scoring='neg_log_loss', cv=5)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)

{'max_iter': 10000, 'penalty': 'l2', 'random_state': 4, 'solver': 'saga'} -0.304834935092


In [175]:
lr = LogisticRegression()
params = {
    'solver': ['liblinear', 'saga'],
    'penalty': ['l1'],
    'random_state': [1,2,3,4,5],
    'max_iter': [10000]
}

gs = GridSearchCV(lr, params, scoring='neg_log_loss', cv=5)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)

{'max_iter': 10000, 'penalty': 'l1', 'random_state': 1, 'solver': 'saga'} -0.248148829504


In [20]:
mlp = MLPClassifier(learning_rate='adaptive', 
                    learning_rate_init=0.005, 
                    max_iter=2000, 
                    random_state=0)
params = {
    'solver':['lbfgs', 'sgd', 'adam']
}

gs = GridSearchCV(mlp, params, scoring='neg_log_loss', cv=5)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)

{'solver': 'sgd'} -0.223207087883


In [29]:
mlp = MLPClassifier(learning_rate='adaptive', 
                    learning_rate_init=0.005, 
                    max_iter=2000, 
                    random_state=0, 
                    solver='sgd')
params = {
    'hidden_layer_sizes': [(100,), (150,), (100, 100,)]
}

gs = GridSearchCV(mlp, params, scoring='neg_log_loss', cv=5)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)

{'hidden_layer_sizes': (150,)} -0.212279414994


In [30]:
mlp = MLPClassifier(learning_rate='adaptive', 
                    learning_rate_init=0.005, 
                    max_iter=2000, 
                    random_state=0, 
                    solver='sgd',
                    hidden_layer_sizes=(150,))
params = {
    'alpha': [10., 5., 2., 1., .8, .5, .2, .1]
}

gs = GridSearchCV(mlp, params, scoring='neg_log_loss', cv=5)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)

{'alpha': 5.0} -0.172329891338


In [11]:
mlp = MLPClassifier(learning_rate='adaptive', 
                    learning_rate_init=0.005, 
                    max_iter=2000, 
                    random_state=0, 
                    solver='sgd',
                    hidden_layer_sizes=(150,),
                    alpha=5.,)
params = {
    'random_state': [0, 1, 2, 3, 4, 5]
}

gs = GridSearchCV(mlp, params, scoring='neg_log_loss', cv=5)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)

{'random_state': 0} -0.172329891338


In [51]:
ada = AdaBoostClassifier(n_estimators=200, learning_rate=1.0, algorithm='SAMME.R', random_state=0)
params = {
    'n_estimators': [100, 200, 400],
    'learning_rate': [1.0, 0.5, 0.1, 0.05]
}

gs = GridSearchCV(ada, params, scoring='neg_log_loss', cv=5)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)

{'learning_rate': 0.05, 'n_estimators': 100} -0.345648843517


In [88]:
et = ExtraTreesClassifier(n_estimators=100, criterion='entropy', random_state=0)

params = {
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2'],
}

gs = GridSearchCV(et, params, scoring='neg_log_loss', cv=5)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
print(gs.best_estimator_.get_params())

{'criterion': 'entropy', 'max_features': 'sqrt'} -0.306649477945
{'bootstrap': False, 'class_weight': None, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': 1, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


In [91]:
gb = GradientBoostingClassifier(n_estimators=400, max_features='sqrt', learning_rate=0.05, random_state=0)
params = {
    'max_depth': [3, 5, 7],
    'max_leaf_nodes': [5, 10, 20]
}

gs = GridSearchCV(gb, params, scoring='neg_log_loss', cv=5)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
print(gs.best_estimator_.get_params())

{'max_depth': 3, 'max_leaf_nodes': 10} -0.146591943787
{'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.05, 'loss': 'deviance', 'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 10, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 400, 'presort': 'auto', 'random_state': 0, 'subsample': 1.0, 'verbose': 0, 'warm_start': False}


In [95]:
rf = RandomForestClassifier(n_estimators=200, criterion='entropy', max_features='sqrt', random_state=0)
params = {
    'max_depth': [3, 5, 7, None],
    #'max_leaf_nodes': [5, 10, 20]
}

gs = GridSearchCV(rf, params, scoring='neg_log_loss', cv=5)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
print(gs.best_estimator_.get_params())

{'max_depth': None} -0.273771535346
{'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': 1, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


In [15]:
lr = LogisticRegression(max_iter=10000, penalty='l1', random_state= 1, solver='saga')
mlp = MLPClassifier(learning_rate='adaptive', 
                    learning_rate_init=0.005, 
                    max_iter=2000, 
                    random_state=0, 
                    solver='sgd',
                    hidden_layer_sizes=(150,),
                    alpha=5.,)

## Training stacking averaged model

In [6]:
from stacking_models_api import StackingAveragedModels
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import log_loss
import xgboost
import lightgbm

In [63]:
sl_base_models_dict = {
    'lr_1': LogisticRegression(max_iter=2000, penalty='l1', random_state= 1, solver='saga'),
    'lr_2': LogisticRegression(max_iter=2000, penalty='l1', random_state= 1, solver='liblinear'),
    'mlp_1': MLPClassifier(learning_rate='adaptive', 
                    learning_rate_init=0.005, 
                    max_iter=2000, 
                    random_state=0, 
                    solver='sgd',
                    hidden_layer_sizes=(150,),
                    alpha=5.,),
    'mlp_2': MLPClassifier(learning_rate='adaptive', 
                    learning_rate_init=0.005, 
                    max_iter=2000, 
                    random_state=0, 
                    solver='adam',
                    hidden_layer_sizes=(150,),
                    alpha=5.,),
    
    'adb_1': AdaBoostClassifier(n_estimators=100, learning_rate=.01),
    'adb_2': AdaBoostClassifier(n_estimators=100, learning_rate=.005),
    'adb_3': AdaBoostClassifier(n_estimators=100, learning_rate=.001),
    'adb_4': AdaBoostClassifier(n_estimators=400, learning_rate=.001),
    'adb_5': AdaBoostClassifier(n_estimators=400, learning_rate=.0005),
    'adb_6': AdaBoostClassifier(n_estimators=400, learning_rate=.0001),
    
    'bg_1': BaggingClassifier(n_estimators=200, max_features=.8, max_samples=.8, random_state=0),
    'bg_2': BaggingClassifier(n_estimators=200, max_features=.7, max_samples=.7, random_state=0),
    'bg_3': BaggingClassifier(n_estimators=200, max_features=.9, max_samples=.9, random_state=0),
    'bg_4': BaggingClassifier(n_estimators=200, max_features=.8, max_samples=.8, bootstrap_features=True, random_state=0),
    'bg_5': BaggingClassifier(n_estimators=200, max_features=.7, max_samples=.7, bootstrap_features=True, random_state=0),
    'bg_6': BaggingClassifier(n_estimators=200, max_features=.6, max_samples=.6, bootstrap_features=True, random_state=0),
    
    'gb_1': GradientBoostingClassifier(n_estimators=200, max_features='sqrt', learning_rate=0.05, 
                                       max_depth=5, max_leaf_nodes=10, random_state=0),
    'gb_2': GradientBoostingClassifier(n_estimators=400, max_features='sqrt', learning_rate=0.05, 
                                       max_depth=3, max_leaf_nodes=5, random_state=0),
    'gb_3': GradientBoostingClassifier(n_estimators=800, max_features='sqrt', learning_rate=0.05, 
                                       max_depth=2, max_leaf_nodes=3, random_state=0),
    'gb_4': GradientBoostingClassifier(n_estimators=200, max_features='sqrt', learning_rate=0.05, 
                                       max_depth=5, max_leaf_nodes=10, subsample=.8, random_state=0),
    'gb_5': GradientBoostingClassifier(n_estimators=400, max_features='sqrt', learning_rate=0.05, 
                                       max_depth=3, max_leaf_nodes=5, subsample=.8, random_state=0),
    'gb_6': GradientBoostingClassifier(n_estimators=800, max_features='sqrt', learning_rate=0.05, 
                                       max_depth=2, max_leaf_nodes=3, subsample=.8, random_state=0),
    
    'rf_1': RandomForestClassifier(n_estimators=200, criterion='entropy', max_features='sqrt', random_state=0),
    'rf_2': RandomForestClassifier(n_estimators=200, criterion='entropy', max_features='sqrt', max_depth=4, random_state=0),
    'rf_3': RandomForestClassifier(n_estimators=200, criterion='entropy', max_features='sqrt', max_depth=8, random_state=0),
    
    'et_1': ExtraTreesClassifier(n_estimators=100, criterion='entropy', max_features='sqrt', random_state=0),
    'et_2': ExtraTreesClassifier(n_estimators=100, criterion='entropy', max_features='sqrt', max_depth=4, random_state=0),
    'et_3': ExtraTreesClassifier(n_estimators=100, criterion='entropy', max_features='sqrt', max_depth=8, random_state=0),
    'et_4': ExtraTreesClassifier(n_estimators=100, criterion='entropy', max_features='sqrt', max_depth=16, random_state=0),
    
    'xgb_1': xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               max_depth=3, 
                               min_child_weight=4, 
                               gamma=.1,
                               n_estimators=500, # slightly more
                               learning_rate=0.01,
                               reg_alpha=.0, # slight more regularization
                               reg_lambda=.1, # slight more regularization
                               random_state=0),
    'xgb_2': xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               max_depth=3, 
                               min_child_weight=4, 
                               gamma=.1,
                               n_estimators=500, # slightly more
                               learning_rate=0.01,
                               reg_alpha=.1, # slight more regularization
                               reg_lambda=.2, # slight more regularization
                               random_state=0),
    'xgb_3': xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               max_depth=3, 
                               min_child_weight=4, 
                               gamma=.2,
                               n_estimators=500, # slightly more
                               learning_rate=0.01,
                               reg_alpha=.0, # slight more regularization
                               reg_lambda=.1, # slight more regularization
                               random_state=0),
    'xgb_4': xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               max_depth=3, 
                               min_child_weight=4, 
                               gamma=.2,
                               n_estimators=500, # slightly more
                               learning_rate=0.01,
                               reg_alpha=.1, # slight more regularization
                               reg_lambda=.2, # slight more regularization
                               random_state=0),
    'lgb_1': lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 boosting_type='dart',
                                 n_estimators=200, # slightly more
                                 min_child_samples=5,
                                 num_leaves=7, 
                                 reg_alpha=.4,  # slight more regularization  
                                 reg_lambda=.2,  # slight more regularization
                                 random_state=0),
    'lgb_2': lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 boosting_type='dart',
                                 n_estimators=200, # slightly more
                                 min_child_samples=5,
                                 num_leaves=7, 
                                 reg_alpha=.45,  # slight more regularization  
                                 reg_lambda=.25,  # slight more regularization
                                 random_state=0),
    'lgb_3': lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 boosting_type='dart',
                                 n_estimators=400, # slightly more
                                 min_child_samples=5,
                                 num_leaves=14, 
                                 reg_alpha=.4,  # slight more regularization  
                                 reg_lambda=.2,  # slight more regularization
                                 random_state=0),
    'lgb_4': lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 boosting_type='dart',
                                 n_estimators=400, # slightly more
                                 min_child_samples=5,
                                 num_leaves=14, 
                                 reg_alpha=.45,  # slight more regularization  
                                 reg_lambda=.25,  # slight more regularization
                                 random_state=0),
}

semi_sl_base_models_dict = {
    #'knn_16': KNeighborsClassifier(n_neighbors=16),
    'knn_32': KNeighborsClassifier(n_neighbors=32),
    'knn_64': KNeighborsClassifier(n_neighbors=64),
    #'knn_128': KNeighborsClassifier(n_neighbors=128)
}

usl_base_models_dict = {
    'kmean_2': KMeans(n_clusters=2),
    'kmean_3': KMeans(n_clusters=3),
    'kmean_4': KMeans(n_clusters=4)
}

In [64]:
sl_base_models_dict = {
    'bg_1': BaggingClassifier(n_estimators=200, max_features=.8, max_samples=.8, random_state=0),
    'gb_1': GradientBoostingClassifier(n_estimators=200, max_features='sqrt', learning_rate=0.05, 
                                       max_depth=5, max_leaf_nodes=10, random_state=0),
    'xgb_1': xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               max_depth=3, 
                               min_child_weight=4, 
                               gamma=.1,
                               n_estimators=500, # slightly more
                               learning_rate=0.01,
                               reg_alpha=.0, # slight more regularization
                               reg_lambda=.1, # slight more regularization
                               random_state=0),
    'xgb_2': xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               max_depth=3, 
                               min_child_weight=4, 
                               gamma=.1,
                               n_estimators=500, # slightly more
                               learning_rate=0.01,
                               reg_alpha=.1, # slight more regularization
                               reg_lambda=.2, # slight more regularization
                               random_state=0),
    'lgb_1': lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 boosting_type='dart',
                                 n_estimators=200, # slightly more
                                 min_child_samples=5,
                                 num_leaves=7, 
                                 reg_alpha=.4,  # slight more regularization  
                                 reg_lambda=.2,  # slight more regularization
                                 random_state=0),
    'lgb_2': lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 boosting_type='dart',
                                 n_estimators=200, # slightly more
                                 min_child_samples=5,
                                 num_leaves=7, 
                                 reg_alpha=.45,  # slight more regularization  
                                 reg_lambda=.25,  # slight more regularization
                                 random_state=0),
    
}

semi_sl_base_models_dict = {
    #'knn_16': KNeighborsClassifier(n_neighbors=16),
    'knn_32': KNeighborsClassifier(n_neighbors=32),
    'knn_64': KNeighborsClassifier(n_neighbors=64),
    #'knn_128': KNeighborsClassifier(n_neighbors=128)
}

usl_base_models_dict = {
    'kmean_2': KMeans(n_clusters=2),
    'kmean_3': KMeans(n_clusters=3),
    'kmean_4': KMeans(n_clusters=4)
}

In [70]:
sam = StackingAveragedModels(sl_base_models_dict=sl_base_models_dict, 
                             #semi_sl_base_models_dict=semi_sl_base_models_dict,
                             #usl_base_models_dict=usl_base_models_dict,
                             meta_model=LogisticRegression(),
                             target_col='is_iceberg',
                             eval_func=log_loss,
                             is_classification=True)

In [71]:
sam.fit(train_X, train_y)


 bg_1
score= 0.204820228475
score= 0.189688933428
score= 0.153336684378
score= 0.195225919493
score= 0.146293384729
Avg score =  0.1778730301

 gb_1
score= 0.198165104448
score= 0.175581511529
score= 0.139609412289
score= 0.176003706616
score= 0.132779120079
Avg score =  0.164427770992

 xgb_1
score= 0.183222899608
score= 0.141988045382
score= 0.118549516528
score= 0.160629102258
score= 0.113158687441
Avg score =  0.143509650243

 xgb_2
score= 0.183108556501
score= 0.142459383075
score= 0.119020607417
score= 0.160492956197
score= 0.113167439307
Avg score =  0.143649788499

 lgb_1
score= 0.174977791653
score= 0.140115903248
score= 0.121351527243
score= 0.152730429658
score= 0.110558125185
Avg score =  0.139946755398

 lgb_2
score= 0.179091875589
score= 0.141311863891
score= 0.11528677174
score= 0.161439997699
score= 0.111992163504
Avg score =  0.141824534484


In [54]:
new_df = sam.get_meta_train_dataframe(get_dummies=True, pca_enabled=False, pca_variance_th=1e-4)

In [55]:
features = new_df.columns.tolist()
features.remove('is_iceberg')

new_X = new_df[features]
new_y = np.array(new_df['is_iceberg']).reshape((new_df.shape[0],))

print(new_X.shape, new_y.shape)

(1604, 49) (1604,)


## Fine-tune meta model: lightGBM

In [40]:
lightgbm.LGBMClassifier().get_params()

{'boosting_type': 'gbdt',
 'colsample_bytree': 1.0,
 'learning_rate': 0.1,
 'max_bin': 255,
 'max_depth': -1,
 'min_child_samples': 10,
 'min_child_weight': 5,
 'min_split_gain': 0.0,
 'n_estimators': 10,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': 0,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 50000,
 'subsample_freq': 1}

In [72]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, objective='binary', random_state=0)
params = {
    'boosting_type': ['gbdt', 'dart'],
    'n_estimators': [10, 100, 200, 400, 800, 1200, 1600],
    #'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.0001]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'boosting_type': 'dart', 'n_estimators': 200} -0.138375143321


In [43]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='goss',
                                 learning_rate=.05,
                                 n_estimators=100)
params = {
    'num_leaves': [2, 3, 7, 15, 31, 63],
    'min_child_samples': [1, 2, 3, 5, 10, 20]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'min_child_samples': 10, 'num_leaves': 31} -0.133331258018


In [44]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='goss',
                                 learning_rate=.05,
                                 n_estimators=100,
                                 num_leaves=31,
                                 min_child_samples=10)
params = {
    'min_split_gain': [.0, .1, .2]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'min_split_gain': 0.0} -0.133331258018


In [45]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='goss',
                                 learning_rate=.05,
                                 n_estimators=100,
                                 num_leaves=31,
                                 min_child_samples=10)

params = {
    'subsample': [1., .8, .6, .4, .2],
    'colsample_bytree': [1., .8, .6, .4, .2]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

LightGBMError: b'cannot use bagging in GOSS'

In [17]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='gbdt',
                                 learning_rate=.01,
                                 n_estimators=400,
                                 num_leaves=3,
                                 min_child_samples=1)

params = {
    'subsample': [.8, .75, .7, .65, .6, .55],
    'colsample_bytree': [1., .98, .95, .93, .9]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'colsample_bytree': 0.98, 'subsample': 0.8} -0.136640756114


In [18]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='gbdt',
                                 learning_rate=.01,
                                 n_estimators=400,
                                 num_leaves=3,
                                 min_child_samples=1, 
                                 colsample_bytree=.98,
                                 subsample=.8)

params = {
    'reg_alpha': [1., .8, .6, .4, .2, .1, .0],
    'reg_lambda': [1., .8, .6, .4, .2, .1, .0]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'reg_alpha': 0.0, 'reg_lambda': 0.0} -0.136640756114


In [19]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='gbdt',
                                 learning_rate=.01,
                                 n_estimators=400,
                                 num_leaves=3,
                                 min_child_samples=1, 
                                 colsample_bytree=.98,
                                 subsample=.8)

params = {
    'reg_alpha': [.075, .05, .025],
    'reg_lambda': [.075, .05, .025]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'reg_alpha': 0.025, 'reg_lambda': 0.025} -0.136680693903


In [26]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='gbdt',
                                 learning_rate=.01,
                                 n_estimators=400,
                                 num_leaves=3,
                                 min_child_samples=1, 
                                 colsample_bytree=.98,
                                 subsample=.8, 
                                 reg_alpha= 0.025, 
                                 reg_lambda= 0.025)

params = {
    'random_state': [0, 1, 2, 3, 4]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'random_state': 1} 0.956982543641


In [21]:
test_df = pd.read_json('Data/test.json')
print(test_df.shape)
test_df.head(5)
test_ids = test_df['id']
test_df = pd.read_csv('Data/pca_projected_266_from_resnet_test.csv')
#test_df.reset_index(drop=True, inplace=True)
print(test_df.columns)
#test_df.sort_index(inplace=True)
test_df.head(5)

(8424, 4)
Index(['f_0', 'f_1', 'f_2', 'f_3', 'f_4', 'f_5', 'f_6', 'f_7', 'f_8', 'f_9',
       ...
       'f_257', 'f_258', 'f_259', 'f_260', 'f_261', 'f_262', 'f_263', 'f_264',
       'f_265', 'is_iceberg'],
      dtype='object', length=267)


,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_257,f_258,f_259,f_260,f_261,f_262,f_263,f_264,f_265,is_iceberg
0,-0.937001,-0.046966,-0.123941,1.657609,0.387005,-0.935819,0.875177,-1.270511,-0.197012,-0.564091,...,-1.455127,-0.459705,0.570348,0.077780,0.078650,1.267042,0.187300,0.923009,0.117212,0.0
1,0.118640,-1.240150,0.447019,-1.328023,0.787310,-1.598214,-1.202442,1.852692,-0.389426,-1.097326,...,3.457100,1.954246,-1.251609,-0.835538,0.614450,-0.270183,1.348282,-3.248190,-3.722244,0.0
2,-1.764784,3.286722,1.683806,-4.562440,1.846337,4.057018,-1.713150,-2.781449,2.058828,-3.035690,...,-9.944841,-10.129503,2.608267,-12.667193,5.982126,4.465105,-5.248517,20.695408,-0.310907,0.0
3,1.349804,1.111627,-0.250717,0.631179,0.484991,-0.054787,1.577700,-0.875416,0.119568,0.265341,...,-0.209830,0.366807,1.484570,0.305957,-1.860656,0.660805,-0.159620,0.538565,0.857225,0.0
4,-0.776327,-0.104925,-1.166807,-0.152499,0.485535,-1.454540,-0.705199,3.206676,-0.383359,-0.126195,...,-2.892168,-2.172888,5.666597,-1.353861,0.659567,0.252173,-4.737852,-0.146233,-1.757885,0.0


In [22]:
features = test_df.columns.tolist()
features.remove('is_iceberg')

test_X = test_df[features]

In [73]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='gbdt',
                                 learning_rate=.01,
                                 n_estimators=400,
                                 num_leaves=3,
                                 min_child_samples=1, 
                                 colsample_bytree=.98,
                                 subsample=.8, 
                                 reg_alpha= 0.025, 
                                 reg_lambda= 0.025)
sam.reset_meta_model(lg_clf)

In [74]:
predictions = sam.predict_proba(test_X)[:,1]
print(predictions)

meta model's training set score=  0.131116933807 

[ 0.01722956  0.3750491   0.02713656 ...,  0.01709143  0.98242059
  0.02713656]


In [25]:
submission = pd.DataFrame()
submission['id'] = test_ids
submission['is_iceberg'] = predictions
submission.to_csv('Submissions/submission_sam_lightgbm.csv', float_format="%.15f", index=False)

> add more regularization

In [97]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=3,
                                 boosting_type='dart',
                                 n_estimators=220, 
                                 min_child_samples=1,
                                 num_leaves=3,
                                 min_split_gain=.0, 
                                 colsample_bytree=.98,
                                 subsample=.7, 
                                 reg_alpha= 0.03, 
                                 reg_lambda= 0.055)
sam.reset_meta_model(lg_clf)

In [98]:
predictions = sam.predict_proba(test_X)[:,1]
print(predictions)

meta model's training set score=  0.105920983433 

[ 0.00933772  0.26000007  0.03301431 ...,  0.01094177  0.98985594
  0.06724654]


D:\Kaggle Competitions\kaggle-Iceberg-Ship-Classifier-\stacking_models_api.py:309: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  out_of_fold_predictions[self.target_col] = self.out_of_fold_predictions[self.target_col]


In [99]:
submission = pd.DataFrame()
submission['id'] = test_ids
submission['is_iceberg'] = predictions
submission.to_csv('Submissions/submission_sam_lightgbm_more_regularization.csv', float_format="%.15f", index=False)

## Fine-tune other meta-model: XGBoost, AdaBoost, Bagging, GradientBoost

In [69]:
xgboost.XGBClassifier().get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': True,
 'subsample': 1}

In [36]:
xg_clf = xgboost.XGBClassifier(n_jobs=4)
params = {
    'booster': ['gbtree', 'gblinear', 'dart'],
    'objective': ['binary:logistic', 'binary:logitraw']
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'booster': 'gbtree', 'objective': 'binary:logistic'} -0.138297540138


In [39]:
xg_clf = xgboost.XGBClassifier(n_jobs=4)
params = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [.3, .1, .05, .01, .005, .001]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'learning_rate': 0.01, 'n_estimators': 500} -0.135785077947


In [68]:
xg_clf = xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               n_estimators=500,
                               learning_rate=0.01)
params = {
    'max_depth': [0, 3, 5, 7, 9],
    'min_child_weight': [.5, 1, 2, 4, 6],
    'gamma': [0, 0.1, 0.2]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)
print(gs.best_estimator_.get_params())

{'gamma': 0.1, 'max_depth': 3, 'min_child_weight': 1} -0.135656785653
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 0.1, 'learning_rate': 0.01, 'max_delta_step': 0, 'max_depth': 3, 'min_child_weight': 1, 'missing': None, 'n_estimators': 500, 'n_jobs': 4, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': None, 'silent': True, 'subsample': 1}


In [41]:
xg_clf = xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               n_estimators=500,
                               learning_rate=0.01,
                               max_depth=3,
                               min_child_weight=1,
                               gamma=.1)
params = {
    'colsample_bytree': [1., .8, .6, .4, .2],
    'subsample': [1., .8, .6, .4, .2]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'colsample_bytree': 0.8, 'subsample': 0.8} -0.13527777042


In [42]:
xg_clf = xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               n_estimators=500,
                               learning_rate=0.01,
                               max_depth=3,
                               min_child_weight=1,
                               gamma=.1)
params = {
    'colsample_bytree': [.9, .85, .8, .75, .7],
    'subsample': [.9, .85, .8, .75, .7]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'colsample_bytree': 0.7, 'subsample': 0.9} -0.134982915664


In [44]:
xg_clf = xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               n_estimators=500,
                               learning_rate=0.01,
                               max_depth=3,
                               min_child_weight=1,
                               gamma=.1,
                               colsample_bytree=.7,
                               subsample=.9)
params = {
    'reg_alpha': [0., .5, 1.],
    'reg_lambda': [1., .5, .1, .05, .01]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'reg_alpha': 0.0, 'reg_lambda': 0.05} -0.134864582512


In [45]:
xg_clf = xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               n_estimators=500,
                               learning_rate=0.01,
                               max_depth=3,
                               min_child_weight=1,
                               gamma=.1,
                               colsample_bytree=.7,
                               subsample=.9,
                               reg_alpha=.0,
                               reg_lambda=.05)
params = {
    'random_state': [0, 1, 2, 3, 4]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'random_state': 0} -0.134864582512


In [47]:
adb = AdaBoostClassifier(n_estimators=100, learning_rate=.001, random_state=0)

params = {
    'n_estimators': [100, 200, 400, 800]
}

gs = GridSearchCV(adb, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'n_estimators': 400} -0.170063332487


In [48]:
adb = AdaBoostClassifier(n_estimators=400, learning_rate=.001, random_state=0)

params = {
    'learning_rate': [.001, .005, .0001, .0005]
}

gs = GridSearchCV(adb, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'learning_rate': 0.001} -0.170063332487


In [49]:
bg = BaggingClassifier(n_estimators=200, random_state=0, n_jobs=4)

params = {
    'bootstrap_features': [True, False],
    'n_estimators': [100, 200, 300, 400,]
}

gs = GridSearchCV(bg, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'bootstrap_features': True, 'n_estimators': 400} -0.239761705774


In [52]:
bg = BaggingClassifier(n_estimators=200, random_state=0, n_jobs=4, bootstrap_features=True)

params = {
    'n_estimators': [400, 800, 1200, 1600, 2400]
}

gs = GridSearchCV(bg, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'n_estimators': 1600} -0.220514189297


In [53]:
bg = BaggingClassifier(n_estimators=1600, random_state=0, n_jobs=4, bootstrap_features=True)

params = {
    'max_samples': [1., .8, .6],
    'max_features': [1., .8, .6]
}

gs = GridSearchCV(bg, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'max_features': 0.6, 'max_samples': 0.6} -0.165277801468


In [54]:
bg = BaggingClassifier(n_estimators=1600, random_state=0, n_jobs=4, bootstrap_features=True)

params = {
    'max_samples': [.7, .6, .5],
    'max_features': [.7, .6, .5]
}

gs = GridSearchCV(bg, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'max_features': 0.7, 'max_samples': 0.5} -0.162197729551


In [57]:
gb = GradientBoostingClassifier(n_estimators=400, max_features='sqrt', learning_rate=0.05, random_state=0)
params = {
    'n_estimators': [200, 400, 800, 1600, 2400],
    'learning_rate': [.1, .05, .01, .005, .001]
}

gs = GridSearchCV(gb, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)
print(gs.best_estimator_.get_params())

{'learning_rate': 0.005, 'n_estimators': 1600} -0.139681349803
{'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.005, 'loss': 'deviance', 'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1600, 'presort': 'auto', 'random_state': 0, 'subsample': 1.0, 'verbose': 0, 'warm_start': False}


In [59]:
gb = GradientBoostingClassifier(n_estimators=1600, max_features='sqrt', learning_rate=0.005, random_state=0)
params = {
    'max_depth': [3, 5, 7],
    'max_leaf_nodes': [5, 10, 20, None]
}

gs = GridSearchCV(gb, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)
print(gs.best_estimator_.get_params())

{'max_depth': 3, 'max_leaf_nodes': 5} -0.139233584438
{'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.005, 'loss': 'deviance', 'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 5, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1600, 'presort': 'auto', 'random_state': 0, 'subsample': 1.0, 'verbose': 0, 'warm_start': False}


In [60]:
gb = GradientBoostingClassifier(n_estimators=1600, max_features='sqrt', 
                                learning_rate=0.005, random_state=0,
                                max_depth=3, max_leaf_nodes=5)
params = {
    'min_impurity_decrease': [0.0, .1, .2]
}

gs = GridSearchCV(gb, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)
print(gs.best_estimator_.get_params())

{'min_impurity_decrease': 0.1} -0.138504706095
{'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.005, 'loss': 'deviance', 'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 5, 'min_impurity_decrease': 0.1, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1600, 'presort': 'auto', 'random_state': 0, 'subsample': 1.0, 'verbose': 0, 'warm_start': False}


In [62]:
gb = GradientBoostingClassifier(n_estimators=1600, max_features='sqrt', 
                                learning_rate=0.005, random_state=0,
                                max_depth=3, max_leaf_nodes=5, 
                                min_impurity_decrease=.1)
params = {
    'min_weight_fraction_leaf': [.0, .1, .2]
}

gs = GridSearchCV(gb, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)
print(gs.best_estimator_.get_params())

{'min_weight_fraction_leaf': 0.1} -0.136178455432
{'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.005, 'loss': 'deviance', 'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 5, 'min_impurity_decrease': 0.1, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.1, 'n_estimators': 1600, 'presort': 'auto', 'random_state': 0, 'subsample': 1.0, 'verbose': 0, 'warm_start': False}


In [65]:
gb = GradientBoostingClassifier(n_estimators=1600, max_features='sqrt', 
                                learning_rate=0.005, random_state=0,
                                max_depth=3, max_leaf_nodes=5, 
                                min_impurity_decrease=.1, min_weight_fraction_leaf=.1)
params = {
    'subsample': [1.,.8, .6, .4, .2]
}

gs = GridSearchCV(gb, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)
print(gs.best_estimator_.get_params())

{'subsample': 0.6} -0.134738336622
{'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.005, 'loss': 'deviance', 'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 5, 'min_impurity_decrease': 0.1, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.1, 'n_estimators': 1600, 'presort': 'auto', 'random_state': 0, 'subsample': 0.6, 'verbose': 0, 'warm_start': False}


In [66]:
gb = GradientBoostingClassifier(n_estimators=1600, max_features='sqrt', 
                                learning_rate=0.005, random_state=0,
                                max_depth=3, max_leaf_nodes=5, 
                                min_impurity_decrease=.1, min_weight_fraction_leaf=.1)
params = {
    'subsample': [.7, .65, .6, .55, .5]
}

gs = GridSearchCV(gb, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)
print(gs.best_estimator_.get_params())

{'subsample': 0.5} -0.134277916917
{'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.005, 'loss': 'deviance', 'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 5, 'min_impurity_decrease': 0.1, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.1, 'n_estimators': 1600, 'presort': 'auto', 'random_state': 0, 'subsample': 0.5, 'verbose': 0, 'warm_start': False}


In [67]:
gb = GradientBoostingClassifier(n_estimators=1600, max_features='sqrt', 
                                learning_rate=0.005, random_state=0,
                                max_depth=3, max_leaf_nodes=5, 
                                min_impurity_decrease=.1, min_weight_fraction_leaf=.1)
params = {
    'subsample': [.5, .475, .45]
}

gs = GridSearchCV(gb, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)
print(gs.best_estimator_.get_params())

{'subsample': 0.5} -0.134277916917
{'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.005, 'loss': 'deviance', 'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 5, 'min_impurity_decrease': 0.1, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.1, 'n_estimators': 1600, 'presort': 'auto', 'random_state': 0, 'subsample': 0.5, 'verbose': 0, 'warm_start': False}


In [73]:
adb = AdaBoostClassifier(n_estimators=400, learning_rate=.001, random_state=0)

bg = BaggingClassifier(n_estimators=1600, random_state=0, n_jobs=4, 
                       bootstrap_features=True, max_features=0.7, max_samples=0.5)

gb = GradientBoostingClassifier(n_estimators=1600, max_features='sqrt', 
                                learning_rate=0.005, random_state=0,
                                max_depth=3, max_leaf_nodes=5, 
                                min_impurity_decrease=.1, min_weight_fraction_leaf=.1,
                                subsample=.5)

xg_clf = xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               n_estimators=500,
                               learning_rate=0.01,
                               max_depth=3,
                               min_child_weight=1,
                               gamma=.1,
                               colsample_bytree=.7,
                               subsample=.9,
                               reg_alpha=.0,
                               reg_lambda=.05)

lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='dart',
                                 n_estimators=200, 
                                 min_child_samples=1,
                                 num_leaves=3,
                                 min_split_gain=.0, 
                                 colsample_bytree=.98,
                                 subsample=.7, 
                                 reg_alpha= 0.0, 
                                 reg_lambda= 0.0)

## Fine-tune final blending model: VotingClassifier

In [72]:
from sklearn.ensemble import VotingClassifier

In [78]:
estimators=[('adb', adb), ('bg',bg), ('gb',gb), ('xg_clf', xg_clf), ('lg_clf', lg_clf)]

vc = VotingClassifier(estimators, voting='soft', n_jobs=4)

params = {
    'weights': [[1,1,3,2,3], [1,1,4,4,4], [1,1,5,4,5]]
}

gs = GridSearchCV(vc, params, scoring='neg_log_loss', cv=5)
gs.fit(new_X, new_y)
print(gs.best_params_ , gs.best_score_)

{'weights': [1, 1, 5, 4, 5]} -0.134376758047


In [79]:
vc = VotingClassifier(estimators, voting='soft', n_jobs=4, weights=[1,1,5,4,5])
sam.reset_meta_model(vc)
predictions = sam.predict_proba(test_X)[:,1]
print(predictions)

D:\Kaggle Competitions\kaggle-Iceberg-Ship-Classifier-\stacking_models_api.py:309: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  out_of_fold_predictions[self.target_col] = self.out_of_fold_predictions[self.target_col]


meta model's training set score=  0.0928196276328 

[ 0.01155164  0.31794963  0.03674663 ...,  0.0146808   0.98699888
  0.06860662]


In [80]:
submission = pd.DataFrame()
submission['id'] = test_ids
submission['is_iceberg'] = predictions
submission.to_csv('Submissions/submission_sam_voting.csv', float_format="%.15f", index=False)